# FIT5196 Data Wrangling Assignment 1
#### Student Name: Gunveen Singh and Mohit Sharma
#### Student ID: 30239028 and 30178835

Date: 08/25/2019

NoteBook Version: 22.0


Environment: Python 3.7.1 and Anaconda 5.7.4 (64-bit)

Libraries used:

* pandas 0.23.4 (for data frame, included in Anaconda Python 3.6) 
* re 2.2.1 (for regular expression, included in Anaconda Python 3.6) 

<b> Notebook Content</b>

<b> 1. [Import the libraries](#1)</b> <br>
<b> 2. [Reading a File](#2)</b><br>
<b> 3. [Compiling a Regular Expression pattern](#3)</b> <br>
<b> 4. [Data Cleaning & Creating lists](#4)</b> <br>
<b> 5. [Creating a DataFrame using pd.DataFrame()](#5)</b> <br>
<b> 6. [Checking Missing Value](#6)</b> <br>
<b> 7. [Saving DataFrame into CSV & JSON format](#7)</b> <br>
<b> 8. [Summary](#8)</b> <br>


<a id="1"></a> 
## 1. Import the libraries

In [1]:
import re
import pandas as pd

<a id="2"></a> 
## 2. Reading a file

In [2]:
file = open(file="Group001.txt", mode='r') #opening the file in read mode
file_content_raw = file.read()
file.close()
text1=re.compile("<\?xml version=\"1\.0\" encoding\=\"UTF\-8\"\?>")
file_content = text1.split(file_content_raw)
while '' in file_content:
    file_content.remove('')
print("No of patents :", len(file_content))    

No of patents : 150


<a id="3"></a> 
## 3. Compiling a Regular Expression pattern

In [3]:
#Witing regular expressions for the desired columns

grant_id = re.compile('file\=\"([U][S]\w\w\d{6})\-\d{8}\.XML\"')   
patent_title = re.compile("<invention-title id=\"\w{5,6}\">(.*?)</invention-title>") 
kind = re.compile("<kind>([A-Z]\d)</kind>")
number_of_claim = re.compile("\<number\-of\-claims\>(\d{1,4})\<\/number\-of\-claims\>")
first_name=re.compile("<first-name>(.*?)</first-name>")
last_name=re.compile("<last-name>(.*?)</last-name>")
citation_by_examiner = re.compile("\<category\>cited by examiner<\/category\>")
citation_by_applicant = re.compile("\<category>cited by applicant\<\/category\>")
claim_text=re.compile("<claim-text>[\s\S<]*</claim-text>")
abstract=re.compile("\<abstract id\=\"abstract\"\>\n\<p id\=\"p\-0001\" num\=\"0000\"\>(.*?)\<\/p\>\n\<\/abstract\>")


# some extra care with claim-text
cleaner = re.compile('<.*?>') 
cleaner2 = re.compile('\n')
cleaner3 = re.compile('\,\,\,')
cleaner4 = re.compile("[\.][\,][\,]")
cleaner5 = re.compile("[\,][\,]")
cleaner6 = re.compile("[\;][\,]")


cleaner = re.compile('<.*?>')
#print(type(file_content[1]))
abst=abstract.findall(file_content[2])
abst=re.sub(cleaner,'',str(abst))
print(abst)

['Apparatus and methods for eye tracking using an optical coherence tomography (OCT) device are disclosed. Such eye tracking may be performed by using information about the shape of the cornea and the corneal apex or using the iris/pupil border obtained using the OCT device.']


In [4]:
# print(grant_id.findall(file_content[2]))
# print(patent_title.findall(file_content[2]))
# print(kind.findall(file_content[1])[0])
# print(number_of_claim.findall(file_content[2]))
# print(list(first_name.findall(file_content[2])))
# print(list(last_name.findall(file_content[2])))
# print(citation_by_applicant.findall(file_content[2]))
# print(citation_by_examiner.findall(file_content[2]))
# print(claim_text.findall(file_content[2]))
# print(abstract.findall(file_content[1]))

# Note this is only pattern object which is used to find all all the variables (So below output is not clean). 
# Using re.findall(Return a list of all non-overlapping matches of pattern in string.) 

Now, We have created Regular Expression. It's time to create a DataFrame but before that we need to take each element from file content. For that we have created empty lists using list comprehension.

<a id="4"></a> 
## 4. Data Cleaning & Creating lists 

In [6]:
gid_list, title_list, kind_list, no_of_claim_list, name_list, applicant_list, examiners_list, claim_list, abstract_list, = ([] for i in range(9))

for line in file_content:
    
    
    gid=grant_id.findall(line) #to find grant_id
    title=patent_title.findall(line) #to find patent_title
    kinds=kind.findall(line) #to find kind
    sclaim=number_of_claim.findall(line) #to find no_of_claims
    
    #to find inventors
    inventors=re.findall("<inventor.*?>[\s\S]*</inventor>",line)
    for person in inventors:
        first=first_name.findall(person)
        last=last_name.findall(person)
    name = [firstName +" "+ lastName for firstName, lastName in zip(first,last)]
    if len(name)==0:
        names="NA"
    else:
        names=name
    
    # here we count citation_by_applicant
    if len(citation_by_applicant.findall(line))==0:
        citation_by_applicants=0
    else:
        citation_by_applicants=len(citation_by_applicant.findall(line)) 
    
    # count for citation_by_examiner
    if len(citation_by_examiner.findall(line))==0:
        citation_by_examiners=0
    else:    
        citation_by_examiners=len(citation_by_examiner.findall(line))   
    
    # For claim_text
    if (len(re.findall("<claim-text>[\s\S<]*</claim-text>",line))==0):
        claim_text=["NA"]
    else:
        claim_text=re.findall("<claim-text>[\s\S<]*</claim-text>",line) 
    
    # For abstract
    abst=abstract.findall(line)
    if len(abst)==0:
        abstracts=["NA"]
    else:    
        
        cleaner = re.compile('<.*?>')
        #print(type(file_content[1]))
        abst=abstract.findall(line)
        abst=re.sub(cleaner,'',str(abst))
        abstracts_2=abst  
        abstracts_1=abstracts_2.replace("['","")
        abstracts=abstracts_1.replace("']","")
        
    # checking length of gid is not equal to 0 then do append to all the lists
    if len(gid)!=0:                             
        gid_list.append(gid[0])
        title_list.append(title[0])
        kind_list.append(kinds[0])
        no_of_claim_list.append(sclaim[0])
        name_list.append(names)
        applicant_list.append(citation_by_applicants)
        examiners_list.append(citation_by_examiners)
        claim_list.append(claim_text[0])
        abstract_list.append(abstracts)
        
# cleaning claim text         
element=0
for items in claim_list:
    claim_list[element]=re.sub(cleaner,'',claim_list[element])
    claim_list[element]=re.sub(cleaner2,',',claim_list[element])
    claim_list[element]=re.sub(cleaner3,',',claim_list[element])
    claim_list[element]=re.sub(cleaner4,'.,',claim_list[element])
    claim_list[element]=re.sub(cleaner5,',',claim_list[element])
    claim_list[element]=re.sub(cleaner6,'; ',claim_list[element])
    element=element+1

# For kind 
Kind1 = [w.replace('P2', 'Plant Patent Grant(with a published application) issued on or after January 2, 2001') for w in kind_list]
Kind2 = [w.replace('B2', 'Utility Patent Grant (with a published application) issued on or after January 2, 2001.') for w in Kind1]
Kind3 = [w.replace('S1', 'Design Patent') for w in Kind2]
Kind4 = [w.replace('B1', 'Utility Patent Grant (no published application) issued on or after January 2, 2001.') for w in Kind3]

<a id="5"></a>
## 5. Creating a DataFrame using pd.DataFrame()  


In [7]:
# Creating data frame
data_frame = pd.DataFrame(
    {'grant_id': gid_list,
     'patent_title': title_list,
     'kind': Kind4,
     'number_of_claims':no_of_claim_list,
     'inventors':name_list,
     'citations_applicant_count':applicant_list,
     'citations_examiner_count':examiners_list,
     'claims_text':claim_list,
     'abstract':abstract_list
    })

<a id="6"></a> 
## 6. Checking Missing Value

In [8]:
data_frame.isnull().sum()

grant_id                     0
patent_title                 0
kind                         0
number_of_claims             0
inventors                    0
citations_applicant_count    0
citations_examiner_count     0
claims_text                  0
abstract                     0
dtype: int64

In [9]:
# First 5 rows of data frame
data_frame.head(5)

,grant_id,patent_title,kind,number_of_claims,inventors,citations_applicant_count,citations_examiner_count,claims_text,abstract
0,USD0854710,Structural post for a screen enclosure,Design Patent,1,[Thomas G. Hendry],95,16,The ornamental design for a structural post fo...,[NA]
1,US10358640,Process for separation of prokaryotic DNA from...,Utility Patent Grant (with a published applica...,9,"[Subhadra Rukmini Jayaraman, Ramya Vutukuru]",4,1,1. A process for separation of prokaryotic DNA...,A process for separating prokaryotic DNA from ...
2,US10357153,"Medical devices, systems, and methods for perf...",Utility Patent Grant (with a published applica...,12,"[Alexander C. Walsh, Paul G. Updike, Richard C...",242,1,1. A method of detecting an eye gaze direction...,Apparatus and methods for eye tracking using a...
3,US10359796,Buffer circuit for enhancing bandwidth of volt...,Utility Patent Grant (no published application...,20,"[Jie-Yu Liao, Chun-Hung Chen, Hsueh-Yi Lee]",0,5,"1. A buffer circuit, comprising:,a first trans...","A buffer circuit includes a first transistor, ..."
4,US10358631,Ketoreductase polypeptides for the preparation...,Utility Patent Grant (with a published applica...,16,"[Oscar Alvizo, Steven J. Collier, Hans-Georg J...",150,1,1. An engineered polypeptide capable of conver...,The disclosure relates to engineered ketoreduc...


In [10]:
data_frame["abstract"][0]

"['A new and distinct variety of grapevine plant named &#x2018;SV30-13-10&#x2019; particularly characterized by its ellipsoidal to ovate shaped, seedless, red skinned berries which are medium-large, sweet, with a fruity Vitis labrusca flavor and which have tender texture. Productivity is high, with spur pruning only needed for a full crop.']"

In [11]:
# print the first row only aftering making DataFrame

# print(data_frame['grant_id'][0])
# print(data_frame['patent_title'][0])
# print(data_frame['kind'][0])
# print(data_frame['number_of_claims'][0])
# print(data_frame['inventors'][0])
# print(data_frame['citations_applicant_count'][0])
# print(data_frame['citations_examiner_count'][0])
# print(data_frame['claims_text'][0])
# print(data_frame['abstract'][0])

<a id="7"></a> 
## 7. Saving DataFrame into CSV & JSON format

In [10]:
# Creating CSV
data_frame.to_csv("Group118.csv",index=False)

In [11]:
# Creating JSON
data_dict={}  # creating a empty dictionary
element=0
for each in gid_list:
    data_dict[each]={"patent_title":title_list[element],
                      "kind":kind_list[element],
                      "number_of_claims":no_of_claim_list[element],
                      "inventors":name_list[element],
                      "citations_applicant_count":applicant_list[element],
                      "citations_examiner_count":examiners_list[element],
                      "claims_text":claim_list[element],
                      "abstract":abstract_list[element]
                      }
    element=element+1

In [12]:
data_dict=str(data_dict).replace("'",'"')  
print(data_dict)

with open('Group118.json', 'w') as file:
    file.write(data_dict)

{"USD0854710": {"patent_title": "Structural post for a screen enclosure", "kind": "S1", "number_of_claims": "1", "inventors": ["Thomas G. Hendry"], "citations_applicant_count": 95, "citations_examiner_count": 16, "claims_text": "The ornamental design for a structural post for a screen enclosure, as shown and described.", "abstract": ["NA"]}, "US10358640": {"patent_title": "Process for separation of prokaryotic DNA from eukaryotic DNA in a whole blood sample", "kind": "B2", "number_of_claims": "9", "inventors": ["Subhadra Rukmini Jayaraman", "Ramya Vutukuru"], "citations_applicant_count": 4, "citations_examiner_count": 1, "claims_text": "1. A process for separation of prokaryotic DNA from eukaryotic DNA in a given whole blood sample infected by pathogen, the process comprising:,extracting the prokaryotic DNA and the eukaryotic DNA from the whole blood sample; contacting the prokaryotic DNA in the whole blood sample with a motif that specifically binds to the prokaryotic DNA and generate

<a id="8"></a> 
## 8. Summary

Completed the task of text parsing of a file in XML format. We extracted all the columns with the given output file and found all the columns to be parsed to the best of our knowladge. Extensive use of regex was done in this process to convert semi-structured data into structured data.